# Background
[Object-oriented programming (OOP)](https://en.wikipedia.org/wiki/Object-oriented_programming) is a programming methodology based on objects. An object can be viewed as the combination of data or state and the operations that can be performed on the data (behaviour). The data items are often called fields or attributes while the operations are known as methods. In addition to objects, OOP frequently makes use of a number of other concepts, including abstraction, encapsulation, composition, inheritance, interfaces, and polymorphism. We will examine how most of these concepts can be utilised in Python, but we won't spend too much time on the theory.

# Topics covered
* What is Object-oriented programming?
* Classes and Objects
    * Defining a class with fields and methods
    * Creating an object
* Encapsulation: protecting your data
* Properties: controlling access to data fields
* Inheritance
* Special Methods
    * Interacting with Notebooks
    * Making your objects callable
* Static Methods

# Classes and Objects
In Python, the basis of OOP is the class. A class is the blueprint from which objects are created, while objects are specific instances of a class. For example, I have a coffee cup and water glass sitting on my desk. Both are unique objects, but they could be considered as instances of a more general class of *cups*. Let's explore these relationships with our first Python class. The simplest possible class would be:

In [ ]:
class Cup:
    pass

We have defined a new class, and can now create objects or instances of it:

In [ ]:
coffee_mug = Cup()
type(coffee_mug)

However, we can't do much with this class. It defines no data and no operations. 

Let's add some now. To make sure the data fields are initialised correctly, we need to add a special `__init__` method. This is an example of a Python [special method](https://docs.python.org/3/reference/datamodel.html#special-method-names). Python calls these methods in specific circumstances, and some useful special methods are covered later in this notebook. For now, it is enough to know that `__init__` is called when a new object is being initialised.

In [ ]:
class Cup:
    def __init__(self, name, capacity, current_volume):
        self.name = name
        self.capacity = capacity
        self.current_volume = current_volume

See the first parameter called `self`? That is how the methods access the specific data for each instance or object. In Python, all (well, almost all - more on that later) class methods need to explicitly specify the `self` reference. This parameter doesn't actually have to be called self. It could be called `this` or `eric` if you wanted, but the Python convention is to use `self`.

It is also considered best-practice to initialise all data fields inside the initialiser method. Some Python tools will even give warnings when you fail to do so.

You access the data stored in an object via dot notation:

In [ ]:
coffee_mug = Cup('coffee mug', 200, 50)
coffee_mug.current_volume

We can now create some objects to represent the drinking items on my desk:

In [ ]:
coffee_mug = Cup('coffee mug', 200, 50)
water_glass = Cup('large water glass', 400, 300)

# It time for another coffee?
print("{0} is {1} percent full".format(coffee_mug.name, coffee_mug.current_volume / coffee_mug.capacity * 100))

OK, now we can define a cup and store some data in it. But surely it would be easier to move the current status report into the class as well. Let's do that now by adding a method:

In [ ]:
class Cup:
    def __init__(self, name, capacity, current_volume):
        self.name = name
        self.capacity = capacity
        self.current_volume = current_volume
    
    def status(self):
        return "{0} is {1:.0f}% full".format(
            self.name,
            self.current_volume / self.capacity * 100)

In [ ]:
coffee_mug = Cup('coffee mug', 200, 50)
water_glass = Cup('large water glass', 400, 300)
coffee_mug.status()

What happens if we try to print `coffee_mug` directly?

In [ ]:
print(coffee_mug)

By default, Python just prints the object type information and a memory address. Not that useful in this case. Let's look at a second special method: `__str__`. This method is called by Python when you pass an object to `print`.

In [ ]:
class Cup:
    def __init__(self, name, capacity, current_volume):
        self.name = name
        self.capacity = capacity
        self.current_volume = current_volume
        
    def __str__(self):
        return "{0} is {1:.0f}% full".format(
            self.name,
            self.current_volume / self.capacity * 100)
    
    def status(self):
        # Keep the status method, but redirect to the __str__ method
        return self.__str__()

In [ ]:
coffee_mug = Cup('coffee mug', 200, 20)
print(coffee_mug)

Much nicer. And definitely time for a new coffee (or a drink of water).

## Hands-on with Simple Classes
Try creating a new Cup definition that provides two additional methods. A `sip` method removes 15ml (by default) from the current volume, and a `refill` method that restores the current volume to full.

In [ ]:
class Cup:
    def __init__(self, name, capacity, current_volume):
        self.name = name
        self.capacity = capacity
        self.current_volume = current_volume
        
    def __str__(self):
        if self.current_volume > 0:
            return "{0} is {1:.0f}% full".format(
                self.name,
                self.current_volume / self.capacity * 100)
        else:
            return "{0} is empty".format(self.name)
    
    def status(self):
        # Keep the status method, but redirect to the __str__ method
        return self.__str__()
    
    def sip(self, amount=15):
        # We guard against reducing the volume below zero, but don't guard against negative amounts.
        self.current_volume = max(0, self.current_volume - amount)
        print('sip')
        # return a reference to self so we can chain method calls
        return self
        
    def refill(self):
        self.current_volume = self.capacity
        # return a reference to self so we can chain method calls
        return self

In [ ]:
coffee_mug = Cup('coffee mug', 200, 200)
print(coffee_mug)
print(coffee_mug.sip().status())
print(coffee_mug.sip().sip().status())

coffee_mug.refill()
print(coffee_mug)

coffee_mug.sip(200)
print(coffee_mug)

# Encapsulation: Protecting Your Data
If we work with the following version of `Cup`, we can quickly reveal a shortcoming relating to data access. First, the class definition:

In [ ]:
class Cup:
    def __init__(self, name, capacity, current_volume):
        self.name = name
        self.capacity = capacity
        self.current_volume = current_volume
        
    def __str__(self):
        if self.current_volume > 0:
            return "{0} can hold {1} ml and currently holds {2} ml ({3:.0f}% full)".format(
                self.name,
                self.capacity,
                self.current_volume,
                self.current_volume / self.capacity * 100)
        else:
            return "{0} is empty".format(self.name)

Now, let's try to create mischief!

In [ ]:
magic_cup = Cup('magic cup', 100, 50)
print(magic_cup)

That makes sense. But that cup is too large ...

In [ ]:
# Make it smaller
magic_cup.capacity = 10
print(magic_cup)

500% full? That is magic.

The problem is that by allowing direct access to the data fields, it is easy to get an object into an inconsistent state. Python provides some tools for reducing but not completely preventing this problem.

## Public, Protected, and Private
Many Object-oriented languages have a concept of levels of access for class data. Typically these are called *public*, *protected*, and *private*. The syntax varies between languages, but the semantics are that public data is freely accessible from outside the class, protected data is only accessible by the class and any subclasses (covered more fully in the inheritance section of this notebook), while private data is not even accessible by subclasses.

Provision for data access levels is part of a concept called *encapsulation* or information hiding.

By default, everything in a Python class is **public**.

We are not talking about trying to hide all data fields all the time, just when it makes sense to do so.

### Implementing Protected Data
The Python language doesn't provide a method to define protected data. But there is a convention to prepend a single underscore to protected fields. Although the language doesn't enforce anything, following this convention makes it clear to users of your class that directly modifying some fields is not part of the design.

In [ ]:
class Cup:
    def __init__(self, name, capacity, current_volume):
        self._name = name
        self._capacity = capacity
        self._current_volume = current_volume
        
    def __str__(self):
        if self._current_volume > 0:
            return "{0} can hold {1} ml and currently holds {2} ml ({3:.0f}% full)".format(
                self._name,
                self._capacity,
                self._current_volume,
                self._current_volume / self._capacity * 100)
        else:
            return "{0} is empty".format(self._name)

In [ ]:
magic_cup = Cup('magic cup', 100, 50)
magic_cup._capacity = 25
print(magic_cup)

So, no data encapsulation at all but at least the intent is clearer.

### Implementing Private Data
Fortunately for fans of encapsulation, Python does provide a data hiding method that is enforced (partly). It does this by applying [name mangling](https://en.wikipedia.org/wiki/Name_mangling#Name_mangling_in_Python) to any member name that is **prefixed with at least two underscores, and suffixed with no more than one underscore**. The mangled name is converted to `_<classname><membername>`. Let's have a look ...

In [ ]:
class Cup:
    def __init__(self, name, capacity, current_volume):
        self.__name = name
        self.__capacity = capacity
        self.__current_volume = current_volume
        
    def __str__(self):
        if self.__current_volume > 0:
            return "{0} can hold {1} ml and currently holds {2} ml ({3:.0f}% full)".format(
                self.__name,
                self.__capacity,
                self.__current_volume,
                self.__current_volume / self.__capacity * 100)
        else:
            return "{0} is empty".format(self.__name)

In [ ]:
magic_cup = Cup('magic cup', 100, 50)
dir(magic_cup)

See the name-mangled fields for capacity, current volume, and name? Now try to modify the size ...

In [ ]:
magic_cup = Cup('magic cup', 100, 50)
print('before: ', magic_cup)
magic_cup.__capacity = 10
print('after:  ', magic_cup)

Note that trying to set the value didn't raise any errors (what do you think actually happened?), but we didn't succeed in putting the object into an inconsistent state.

However, the mangled names are still actually accessible if you really try, so everything really is public after all. This is a really bad idea though.

In [ ]:
magic_cup._Cup__capacity = 10
print(magic_cup)

# Elegant Encapsulation with Properties
We have looked at some methods of data encapsulation through naming conventions and name mangling. But what if we want users of our `Cup` class to have a simple but robust way of adjusting the capacity? A way that includes error checks but still looks likes direct access to a public field. I want to be able to write `my_cup.capacity = 50` but still avoid inconsistencies. Fortunately for us Python provides a way to do just that: the `@property` decorator.

Using decorators requires the class implementer to write more code, so they needn't be used on everything. Name mangling is sufficient for fields that don't need any public access, and if you have no need of any additional logic when a value is accessed, then a public field will be fine. Additionally, because public fields and properties look the same from the outside, you can easily start with public data and then switch to properties later without having to change the client code.

Let's see how to write a new `Cup` class with a capacity property.

In [ ]:
class Cup:
    def __init__(self, name, capacity, current_volume):
        # There is no reason for name to be private or a property. It's just a name.
        self.name = name
        self._current_volume = current_volume
        
        # When using properties, the property should be called from other class methods.
        # It is a good practice to only access the private backing field from the property 
        # accessors themselves.
        self.capacity = capacity
        
    # First we define a normal method for returning the property value.
    # The name of the method is the name we want for the public property.
    @property
    def capacity(self):
        # In this case we are returning the private value,
        # but you can also define properties that return derived values.
        return self.__capacity
    
    # Now we define a second method with the same name. 
    # This method contains the logic for setting the property value.
    @capacity.setter
    def capacity(self, capacity):
        # Store the new capacity
        # This might not be the best approach, but for simplicity treat
        # negative values as 0
        self.__capacity = max(0, capacity)
            
        # Spill any excess liquid
        self._current_volume = min(self._current_volume, self.capacity)
        
    def refill(self):
        self._current_volume = self.capacity
        
    def __str__(self):
        if self._current_volume > 0:
            return "{0} can hold {1} ml and currently holds {2} ml ({3:.0f}% full)".format(
                self.name,
                self.capacity,
                self._current_volume,
                self._current_volume / self.capacity * 100)
        else:
            return "{0} is empty".format(self.name)

In [ ]:
coffee = Cup("Espresso!", 80, 40)
print(coffee)

# I don't want an espresso. Make it a long black
coffee.name = "long black"
coffee.capacity = 200
coffee.refill()
print(coffee)

# Actually, can I have an espresso after all?
coffee.name = "make up your mind!"
coffee.capacity = 80
print(coffee)

# Uh oh, I dropped and smashed the cup
coffee.capacity = -1
print(coffee)

That's a bit nicer. Just as simple to use as public data, but increased safety. Hooray!

## Read-only Properties and Monkey Patching
Let's look at adding a read-only property that returns a dynamically calculated value. Also, I am getting tired of pasting the entire class definition each time, so lets look at a technique called *monkey patching*. 

What is a monkey patch? First, unlike some other languages, classes in Python are just objects. *Mutable* objects. So we can change the class definition after the fact. A monkey patch is simply the name given to code that modifies the definition of a class after the class has been defined. It's not a good idea to do this all the time. Code can get hard to understand when you can't trust the class definition. But it is something you will see in existing code, and used wisely monkey patching is a valuable technique.

In [ ]:
# First, define the function that will become the new class method
@property
def percent_full(self):
    if self.capacity > 0:
        return self._current_volume / self.capacity * 100
    else:
        return 0

# now monkey patch the new property into the existing Cup class
Cup.percent_full = percent_full

# Let's try it out
patched_coffee = Cup("patched", 200, 150)
print(patched_coffee)
print(patched_coffee.percent_full)

It works. But is it read-only?

In [ ]:
try:
    patched_coffee.percent_full = 40
except:
    print("Yep, can't set the property")

# Inheritance
An aspect of OOP that we have yet to look at is inheritance. Inheritance allows you to derive a new class from an existing one to build up a hierarchy of related classes that differ in key ways. Typical examples include building a class hierarchy to draw shapes on a screen. You may have a `Shape` class with all the common data and behaviours (position, color etc), and then derive sub-classes to define specialised shapes: `Circle`, `Square` etc. The sub-classes add the unique data and behaviours required for just that one type of shape.

I don't want to get drawn too deeply into the principles, benefits, and risks of inheritance hierarchies here. The goal is to look at how to implement inheritance in Python. Whether building a class hierarchy is a good idea for your application is a much harder question to answer.

In [Learn Python the Hard Way](https://learnpythonthehardway.org/book/ex44.html) inheritance is described as an evil forest full of traps, dangers, and complexity. I highly recommend reading that linked chapter of Learn Python the Hard Way. It describes quite clearly some of the complexities that inheritance can lead to, and an alternative solution known as composition or aggregation. Be that as it may, inheritance is everywhere in Python, so it pays to understand the basics. 

## Terminology
* A class can inherit attributes and methods from another class, called the ***superclass*** or ***parent class***.
* A class which inherits from a superclass is called a ***subclass*** or ***child class***.

## New Style Objects vs Old Style Objects
What was that about inheritance being everywhere in Python? In Python 3, all classes implicitly inherit from a special system class called `object`. The `object` class provides core functionality that is common to all Python objects. So all the `Cup` classes we have created have already been using inheritance implicitly. If you paid attention to the output from `dir(Cup)`, you will have noticed a whole bunch of items that we didn't define. They come from `object`.

In [ ]:
dir(Cup)

In [ ]:
dir(object)

So what about new-style and old-style objects? In Python 2.2, a new type of object was introduced. This *new style* object had different functionality and a different implementation. The original object implementation was labeled *old style*. To retain  backwards compatibility, old style objects remained the default in Python 2. To use new style objects in Python 2, you had to explicitly inherit from the new `object` class:

```python
    def OldStyleIn2_NewStyleIn3:
        pass
        
    def NewStyleIn2and3(object):
        pass
```

In Python 3, old style objects have been removed completely. You always get a new style object regardless of whether you explicitly or implicitly inherit from `object`. One consequence of this is that in order to write universal code that runs with Python 2 and Python 3, it is a good practice to explicitly inherit from `object`.

## The Three Basic Uses of Inheritance
I mentioned that the [Learn Python the Hard Way chapter on inheritance](https://learnpythonthehardway.org/book/ex44.html) is well worth reading. In fact, it is hard to think of a clearer description of the 3 basic uses of inheritance. I will therefore summarise part of the chapter here, but please follow up with the book itself for the second part of the discussion on replacing inheritance with composition.

When creating a subclass, there are typically three basic goals:
1. Actions on the child imply an action on the parent.
2. Actions on the child override or replace an action on the parent.
3. Actions on the child modify the action on the parent.

Let's look at each of these in turn. 

### Implied Actions on the Parent
This happens when a subclass inherits an operation as-is, without making any modifications.

In [ ]:
class Parent(object):
    def implicit(self):
        print("{0} calling Parent implicit()".format(type(self)))
        
class Child(Parent):
    pass

dad = Parent()
son = Child()

dad.implicit()
son.implicit()

You can see that `Parent.implicit` is called regardless of whether the method was called on an object of type `Parent` or of type `Child`. Calling `implicit()` on `son` implicitly called the method from the superclass.

### Overriding Actions on the Parent
Sometimes the reason we create a subclass is so the subclass can do things differently. Replacing the functionality of a superclass is called *overriding* that functionality.

In [ ]:
class Parent(object):
    def override(self):
        print("{0} calling Parent override()".format(type(self)))
                
class Child(Parent):
     def override(self):
        print("{0} calling Child override()".format(type(self)))

dad = Parent()
son = Child()

dad.override()
son.override()

So in this case, the version of the `override()` method depends on the type of the object making the call. Call `override()` on a `Parent` type and you get the `Parent.override()` method. Call `override()` on a `Child` type and you get the `Child.override()` method.

In the original example of defining shape classes, overriding could be used to define the `draw()` methods, allowing each shape to draw itself in a different way.

### Modifying the Actions of a Parent

In [ ]:
class Parent(object):
    def altered(self):
        print("{0} calling Parent altered()".format(type(self)))
        
class Child(Parent):

    def altered(self):
        print("Child, doing some work before calling Parent altered()")
        super().altered()
        print("Child, doing some work after calling Parent altered()")

dad = Parent()
son = Child()

dad.altered()
print()
son.altered()

Here we can see the subclass doing work both before and after calling the superclass. This code also introduces the `super()` function. `super()` hides some complex functionality, particularly when multiple inheritance is being used, but the gist is that it resolves to the next superclass. In a linear hierarchy like we have here, the next superclass is the immediate parent class. In Python 2, you need to supply the current class name and the self reference, so `super()` becomes `super(Child, self)`.

Although it might be regarded as a [code smell](https://en.wikipedia.org/wiki/Code_smell), it is possible to explicitly call a specific superclass in the hierarchy.

In [ ]:
class Grandparent(object):
    def do_something(self):
        print("{0}: Grandparent.do_something()".format(type(self)))
        
class Parent(Grandparent):
    def do_something(self):
        # override Grandparent.do_something()
        print("{0}: Parent.do_something()".format(type(self)))
        
class Child(Parent):
    def do_something(self):
        # Do something ourselves
        print("{0}: Child.do_something()".format(type(self)))
        # Then call Grandparent.do_something(), bypassing the Parent override
        # This works because a class is also the name of an object, so read this line as
        # "call the do_something method on the class object called Grandparent, and pass self as the data reference"
        Grandparent.do_something(self)
        
    def do_something_with_super(self):
        # Do something ourselves
        print("{0}: Child.do_something_with_super()".format(type(self)))
        super().do_something()
        
child = Child()
child.do_something()
print()
child.do_something_with_super()

## __init__: the most common use of super()
I want to finish this section on inheritance with the most common use of `super()`: making sure all classes in a hierarchy are correctly initialised. Consider a new class hierarchy that extends our `Cup` class into a more general `Container` superclass and a `Cup` subclass. Containers hold things but you can't drink from them. Cups do everything a container does while adding the ability to drink their contents. For the sake of brevity, we omit any error checks, methods, and properties that might make sense for these classes but are not required to illustrate the correct initialisation of superclasses.

In [ ]:
class Container(object):
    def __init__(self, name='container', current_volume=0, capacity=1000):
        self.name = name
        self.current_volume = current_volume
        self.capacity = capacity
        
    def __str__(self):
        return '{0} holds {1}/{2}'.format(self.name, self.current_volume, self.capacity)
    
class Cup(Container):
    # Override the superclass __init__ so we can define different default values
    def __init__(self, name='cup', current_volume=0, capacity=200):
        # Now call __init__ on the superclass, passing in the required data
        super().__init__(name, current_volume, capacity)
        
    def drink(self, amount=10):
        print('Drinking ', amount)
        self.current_volume = max(self.current_volume - amount, 0)
        
container = Container()
print(container)

cup = Cup(current_volume=150)
cup.drink()

print(cup)

# you can drink from a cup, but not a container?
container.drink()

In the above code cell, calling `print(cup)` is an example of substitutability or [subtype polymorphism](https://en.wikipedia.org/wiki/Subtyping). Any valid code written for a superclass (`Container`) should be expected to execute when a subclass is substituted. When calling `print`, we didn't have to check what type of object we had, so long as it implemented `__str__`. Since a default version of `__str__` is provided by the `object` class, we can substitute any class.

However, not every class defines a `drink` method. Not even `Container` defines it. So if we want to call `drink` we need to make sure that we have the right type of object. One way to do this is with `isinstance`, an example of [*look before you leap (LBYL)*](https://docs.python.org/3/glossary.html#term-lbyl) coding style:

In [ ]:
if isinstance(container, Cup):
    container.drink()
else:
    print("Can't drink")

Some would argue that this is not pythonic, since it breaks the principle of [*duck typing*](https://en.wikipedia.org/wiki/Duck_typing) - the idea that if something looks like a duck then it is a duck. In Python this means that often we don't actually care what type of objects we have so long as they define the methods we want to call (walk like a duck). 

So if we wanted to drink from something without checking for an explicit type, one way might be to use an [*easier to ask for forgiveness than permission (EAFP)*](https://docs.python.org/3/glossary.html#term-eafp) coding style:

In [ ]:
try:
    container.drink()
except:
    print("Can't drink")

For one thing, this means that the code won't break if the class hierarchy gets changed so that more than one subclass (or the `Container` class itself) supports drinking. In fact, the second version will work for anything that provides a `drink` method, regardless of whether it is related to the `Container` class or not.

# Special Methods
Recall that [special methods](https://docs.python.org/3/reference/datamodel.html#special-method-names) are called by Python in specific circumstances. We have already seen the `__init__` method that is called when creating a new object, and the `__str__` method called when an object is passed to `print`. We won't explore the full set of special methods here, just some of the more commonly useful ones.

## \__init__
When creating a new object, `__init__` is called after `__new__` but before returning the object to the caller. Typically init is used to initialise the object. Also, if the class inherits from anything other than `object`, the init definition should also call init on the superclass.

## \__str__
Called by `str(object)` and `print(object)`, this method should return a string that represents a *pretty-printed* or informative description of the object. The default implementation calls `__repr__`.

## \__repr__
Called by the `repr()` built-in function to retrieve the official string representation of an object. The Python documentation suggests that the string represent valid Python expression that could be used to recreate the object. Sometimes this can be done, and sometimes it can't. Note that the default implementation for custom classes just returns the class name and the memory address of the object. This is useful for debugging but can't be used to recreate the specific object.

In [ ]:
repr(cup)

In [ ]:
class Container(object):
    def __init__(self, name='container', current_volume=0, capacity=1000):
        self.name = name
        self.current_volume = current_volume
        self.capacity = capacity
        
    def __str__(self):
        return '{0} holds {1}/{2}'.format(self.name, self.current_volume, self.capacity)
    
    def __repr__(self):
        return 'Container(name={0}, current_volume={1}, capacity={2})'.format(
            self.name,
            self.current_volume,
            self.capacity)
    
c = Container()
print(str(c))
print(repr(c))

## \_repr\_html_
Strictly speaking, `_repr_html_` is not a Python special method (note the single underscores - core special methods all use double underscores). This method is [supported by Jupyter notebooks](http://ipython.readthedocs.io/en/stable/config/integrating.html?highlight=_repr_) to display a richer version of your objects. This is how pandas data frames manage to look so pretty - they provide a `_repr_html_` method that returns a nicely formatted HTML table. In addition to HTML, other methods are supported for formats such as svg, png, jpeg, javascript, and latex. If these methods don't exist, or if they return `None`, then the output from `__repr__` is used instead.

In [ ]:
# Monkey patch into the previous Container definition
def _repr_html_(self):
    return '<span style="color:green"><h1>{0}</h1></span><span style="color:blue">{1}/{2}</span>'.format(
        self.name, self.current_volume, self.capacity)

Container._repr_html_ = _repr_html_

Container()

## \__call__
Called when the instance is *called* as a function; if this method is defined, `x(arg1, arg2, ...)` is a shorthand for `x.__call__(arg1, arg2, ...)`.

This special method essentially lets you call an object as though it was a function. Let's look at an interesting use of call to define a decorator. I am leaving out comments this time. See if you can figure out how it all works.

In [ ]:
class FunctionWrapper(object):

    def __init__(self, arg1, arg2=5):
        self.arg1 = arg1
        self.arg2 = arg2
        
    def __str__(self):
        return 'arg1={0}, arg2={1}'.format(self.arg1, self.arg2)

    def __call__(self, f):
        def new_f():
            print("Entering", f.__name__)
            print(self)
            f()
            print("Leaving", f.__name__)
        return new_f


@FunctionWrapper("Eric the half a bee")
def hello():
    print("Hello")
    
hello()

## \__bool__
Called when an object is used where a boolean value is expected, or if passed to the built in `bool` function.

In [ ]:
class Container(object):
    def __init__(self, name='container', current_volume=0, capacity=1000):
        self.name = name
        self.current_volume = current_volume
        self.capacity = capacity
        
    def __str__(self):
        return '{0} holds {1}/{2}'.format(self.name, self.current_volume, self.capacity)
    
    def __bool__(self):
        "Returns True if the container contains something, or False if it is empty."
        return self.current_volume > 0
    
class Cup(Container):
    # Override the superclass __init__ so we can define different default values
    def __init__(self, name='cup', current_volume=0, capacity=200):
        # Now call __init__ on the superclass, passing in the required data
        super().__init__(name, current_volume, capacity)
        
    def drink(self, amount=10):
        self.current_volume = max(self.current_volume - amount, 0)

        
coffee = Cup("coffee", current_volume=50)

# note how we are relying on the __bool__ behaviour inherited from the superclass
while coffee:
    print('drink some coffee')
    coffee.drink()

print('all gone')

## Comparison Methods
Python supports a number of special methods that all deal with how comparisons (greater than, equals, less than, etc) should be defined for a class. For details of their implementation I refer you to the [Python documentation](https://docs.python.org/3/reference/datamodel.html#object.__lt__).

# Static Methods
So far, all the methods we have defined have had `self` as the first argument. But what if you have a method that never needs to access data or methods via `self`? You could define a `self` argument and then not use it. But users of your class will still need to have an object to call the method. Consider that you are creating a class to represent email addresses, and you want a method that checks whether a given string represents a valid email address:

In [ ]:
class NotVeryUsefulEmail(object):   
    # To call this you need an object
    # And yes, I know this is a pointless and terrible implementation ...
    def is_valid(self, email):
        parts = email.split('@')
        if len(parts) == 2:
            return len(parts[0]) > 0 and len(parts[1]) > 0
        return False
    
my_email = "eric@monty.python"

# To call the not very useful validation method, we need an object
validator = NotVeryUsefulEmail()
print(validator.is_valid(my_email))

# Or we need to jump through strange hoops
print(NotVeryUsefulEmail.is_valid(None, my_email))

Definitely not very useful. In OOP jargon, a method that doesn't require a reference to an object instance (`self` in Python) is called a *static method*. Python provides a `staticmethod` decorator for just this purpose:

In [ ]:
class Email(object):   
    # The staticmethod decorator indicates that this method doesn't require a self reference
    @staticmethod
    def is_valid(email):
        parts = email.split('@')
        if len(parts) == 2:
            return len(parts[0]) > 0 and len(parts[1]) > 0
        return False
    
print(Email.is_valid('john@python.com'))
print(Email.is_valid('eric@'))

# References
* [Learn Python the Hard Way](https://learnpythonthehardway.org)
* [Python Special Methods](https://docs.python.org/3/reference/datamodel.html#special-method-names)